### 改变对象的字符串显示
重新定义__str__() 和__repr__()方法,对应的函数str()和repr()

In [1]:
class Point:

    def __init__(self) -> None:
        self.x=1.0
        self.y=2.0
    
    def __str__(self) -> str:
        return f'point {self.x} and {self.y}'
    
    def __repr__(self) -> str:
        return self.__str__()
    

p=Point()
repr(p)

'point 1.0 and 2.0'

### 自定义字符串格式化
调用format方法会调用类的__format__方法

In [4]:
from datetime import *
d=date(2012,11,11)

print(d)

2012-11-11


### 上下文管理器
通过实现__enter__()和__exit__()生成上下文管理器

### 节约类空间
通过定义__slots__=[''],python会使用更加紧凑的内存布局,注意python很多特性都是基于字典实现的,定义类slots就不在支持普通类型

### 封装属性名
python通过方法命名来规约达到封装效果,单下划线开头表示内部实现,使用双下滑线会变成私有命名比如__private +> _B__private

### 创建可管理对象属性
通过@property装饰器来管理对象属性.property的关键特征它看上去跟普通的attribute没什么两样,但是访问它的时候自动触发getter,setter,delete方法.三个装饰修饰的方法名字应该相同

> 一个property属性其实就是一系列相关绑定方法的集合,可以直接在类中绑定`property(fget,fset,fdel)`

In [8]:

class Person:

    def __init__(self,name) :
       self._name=name

    @property 
    def name(self):
        return self._name

    @name.setter
    def name(self,name):
        if not isinstance(name,str):
            raise TypeError('must be str')
        self._name=name

    @name.deleter
    def name(self):
        raise AttributeError('can‘t del')
    
    def set_age(self,a):
        print('set')
        self._age=a

    def get_age(self):
        return self._age

    age=property(get_age,set_age)
p=Person('harden')
p.age=1

Person.name.fget
Person.name.fset
Person.name.fdet

set


<function __main__.Person.name(self)>

### 调用父类方法

调用父类的方法一般是使用supper,避免A.__init___(self).使用supper还可以让类起到混合类作用.

> 为了实现继承,python会在MRO上从左到右开始找基类,直到找到一个匹配这个属性对象为止,MRO列表通过C3算法来实现的,三条准则.1.子类会优先于父类被检查;2.多个父类会根据它们在列表中的顺序被检查;3.如果对下一个类存在合法选择,选择第一个父类;

> 当使用supper时,python会在MRO列表上基础搜索下一个类,只要每个重定义的统一使用supper那么控制流会遍历完整个MRO列表,每个方法只会被调用一次

In [8]:
class A:
    _filed=[]
    def __init__(self) -> None:
        print(self._filed,self)

    def max_f(self):
        print(A.__name__)

# 混合类
class Proxy(A):

    def max_f(self):
        super().max_f()
        print(Proxy.__name__)

class B(Proxy,A):
    _filed=['a','b']

    def __init__(self) -> None:
        super().__init__()
b=B()
print(B.__mro__)
b.max_f()

['a', 'b'] <__main__.B object at 0x106b49190>
(<class '__main__.B'>, <class '__main__.Proxy'>, <class '__main__.A'>, <class 'object'>)
A
Proxy


### 子类扩展property


In [55]:
class A:

    @property
    def a(self):
        print('get A')
        return '1'
    
    @a.setter
    def a(self,val):
        print('set A')

    @a.deleter
    def a(self):
        print('del A')

class SubA(A):
    
    # 重新 覆盖A类的a
    @property
    def a(self):
       super().a
       return 'a' 

    @a.setter
    def a(self,val):
        # property属于类属性,supper(type,type_or_obj)
        super(SubA,SubA).a.__set__(self,val)
        print('sub A')

# 覆盖单个
class SubA1(A):
    @A.a.getter
    def a(self):
        print('SubA1')
        return '_14'

a=SubA()
a.a=2



set A
sub A
<class 'str'>
<property object at 0x1080afe20> <property object at 0x1080afce0>


### 创建新的类或实例属性
用描述器在类的属性上检查功类型
> 描述器就是一个实现了三个核心饿属性访问操作的类(get,set,delete),分别对应的__get__,__set__,__delete__三个特殊方法.使用描述器需要将它作为实例放到类的定义中
> https://docs.python.org/zh-cn/3/howto/descriptor.html#primer

### 延迟计算
利用装饰器的get来实现

In [8]:
class TypeCheck:
    def __init__(self,name,type_) :
        self.name=name
        self.type_=type_

    def __get__(self,instance,cls):
        print(cls)
        if not instance:
            return self
        return instance.__dict__[self.name]

    def __set__(self,instance,value):
        if not isinstance(value,self.type_):
            raise TypeError('not support type')
        instance.__dict__[self.name]=value

    def __delete__(self,instance):
        del instance.__dict__[self.name]

class Point:
    x=TypeCheck('x',int)
    y=TypeCheck('y',int)

    def __init__(self,x,y):
        self.x=x
        self.y=y

p=Point(1,2)
p.x


# 利用装饰器来实现延迟加载
class LazyLoader:

    def __init__(self,func):
        self.func_=func

    def __get__(self,instance,cls):
        if not instance:
            return self
        val=self.func_(instance)
        setattr(instance,self.func_.__name__,val)
        return val

class F:

    @LazyLoader
    def a(self):
        return 1*2
    
f=F()
f.a

<class '__main__.Point'>


2

### 简化数据结构的初始化

In [17]:
class Stucture:
    _filed={}

    def __init__(self,*args,**kwargs): 

        if len(args)>len(self._filed):
            raise TypeError('len')

        # self._filed子类找     
        for k,v in zip(self._filed,args):
            setattr(self,k,v)

       
        diff=self._filed&kwargs.keys()
        for name in diff:
            # 为什么不使用__dict__.update.如果类使用__slots__就没有用了
            setattr(self,name,kwargs.pop(name))
            
        # keyword必须清空
        if kwargs:
            raise TypeError('int') 


class Person(Stucture):
    _filed={'name','age'}

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

p=Person('harden',age='13')
p.age


'13'

### 定义接口或者抽象基类
只需要继承abc.ABC
> 尽管ABC可以很方便的做类型检查,但在代码中最好必要过多使用它,因为python本质上一门动态编程语言,其目的为了更加灵活

In [ ]:
from abc import ABC,abstractmethod

class AbcstractClass(ABC):
    @abstractmethod
    def fun(self):
        pass


### 数据模型的类型约束
8.13

### 实现自定义容器
使用collections下的抽象基类

### 属性的代理访问
利用__getattr__方法,__getattr__方法是在找不到attribute属性时被调用


In [19]:
class ReadDo:

    def func_a(self):
        print('func_a')

    def func_b(self):
        print('func_b')

    def func_c(self):
        print('func_c')

class Wrapper:

    def __init__(self,obj):
        self._obj=obj

    def __getattr__(self,name):
        return getattr(self._obj,name)

w=Wrapper(ReadDo())
w.func_a()

func_a


### 在类中定义多个构造器
使用@classmethod装饰器来装饰一个方法,并且该方法的第一个参数是cls对象

### 创建不调用init方法的实例
可以通过class.__new__(class)来创建实例,以此来绕过__init__方法

> 在反序列化对象或实现某个方法构造函数时可能绕过init
> 使用这种方法创建实例时,最好不要用__dict__访问数据,以防类使用__slots__

In [23]:
class Date:

    def __init__(self,year,mouth):
        self.year=year
        self.mouth=mouth
    
    @classmethod
    def now(cls):
        import time
        t=time.localtime()
        return cls(t.tm_year,t.tm_mon)

d=Date.now()

# __new__绕过__init__
d1=Date.__new__(Date)


### 利用Mixins扩展类功能

In [2]:
# 扩展映射对象,给他添加日志

class LonggerMixins:
    # 混入类没有实例变量
    __slots__=[]

    def __getitem__(self,key):
        print('name -> ',key)
        return super().__getitem__(key)
    
    def __setitem__(self,key,value):
        return super().__setitem__(key,value)
    
    def __delitem__(self,key):
        return super().__delitem__(key)

class MyDict(LonggerMixins,dict):
    pass

d=MyDict([('1','2')])
d['1']

name ->  1


'2'

### 实现状态对象


```python
class Connection:
    """普通方案，好多个判断语句，效率低下~~"""

    def __init__(self):
        self.state = 'CLOSED'

    def read(self):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('reading')

    def write(self, data):
        if self.state != 'OPEN':
            raise RuntimeError('Not open')
        print('writing')

    def open(self):
        if self.state == 'OPEN':
            raise RuntimeError('Already open')
        self.state = 'OPEN'

    def close(self):
        if self.state == 'CLOSED':
            raise RuntimeError('Already closed')
        self.state = 'CLOSED'
```

In [ ]:

class Connection:

    def __init__(self):
        ... # close

    def read(self):
        self._state.read()

    def write(self, data):
        self._state.write(data)
    
    def open(self):
        self._state.open()
    
    def close(self):
        self._state.close()
    
    def new_state(self,state):
        self._state=state

class State:
    @staticmethod
    def read(con):
        raise NotImplementedError()        
    
    @staticmethod
    def write(con):
        raise NotImplementedError()        
    
    @staticmethod
    def open(con):
        raise NotImplementedError()        

    @staticmethod
    def close(con):
        raise NotImplementedError()        
class OpenState:

    @staticmethod
    def read(con):
        raise NotImplementedError()        

    @staticmethod
    def write(con):
        raise NotImplementedError()        

    @staticmethod
    def open(con):
        con.new_state(OpenState)

    @staticmethod 
    def close(con):
        raise NotImplementedError()        

### 通过字符串调用对象方法
- 通过getattr()方法:调用一个方法实际上两部独立操作,第一步查找属性,第二部函数调用
- operator.methodcaller()创建一个可调用对象,并同时提供必要参数,然后将实例传递给它

8.21
8.22

In [6]:
import operator
class Foo:

    def add(self,a,b):
        return a+b

f=Foo()
print(getattr(f,'add')(1,2))
print(operator.methodcaller('add',1,2)(f))

3
3


### 循环引用数据结构内存管理
比如,循环引用数据结构就是一个树形结构,双亲节点都有指针指向孩子节点,孩子节点又返回来指向双亲节点.这种情况下使用weakref库的弱引用

> 循环引用是python中很棘手的问题,因为正常的垃圾回收机制不适应这种情形.python使用引用计数,当对象引用数据变成0时才会被删除,这种情况下可以使用gc.colletion()来处理

> 弱引用就是一个对象指针,他不会增加它引用计数


### 让类支持比较操作
想让某个类的实例支持标准的运算,不想实现一大丢特殊方法.可通过functools.total_ordering来简化处理,使用它来装饰一个类,只需定义一个__eq__方法,和其(__lt__,__le__,__gt__,__or__,__ge__)中的一个即可,装饰器会自动为它们填充

### 创建换成实例
使用weakref.WeakValueDictionary()